# MATH 405 - Numerical Methods for Differential Equations

# MATH 607 - Topics in Numerical Analysis

[[Instructor: Christoph Ortner]](http://www.math.ubc.ca/~ortner/)   [[course page]](https://github.com/cortner/math405_2022)



# L05 - Floating Point Arithmetic

# L05 - Floating Point Arithmetic

The content of this lecture is not Julia specific but applies to all languages! The topic may seem dry and tedious, and as a matter of fact we want to minimize our engagement with it. We can afford to do so because of people like Kahan who have spent their lives ensuring we have robust floating point arithmetic available now. But it is important that we at least  have a basic understanding.

#### Further Reading 

* Michael Overton, Numerical Computing with IEEE Floating Point Arithmetic, SIAM 
* Nicholas Higham, Accuracy and Stability of Numerical Algorithms, SIAM

### Integers

$$ 
    \dots, -3, -2, -1, 0, 1, 2, 3, \dots 
$$

Integers are, by default, of type `Int = Int64`. They are represented as 64 bit (8 byte) in memory (base - 2). E.g. 

In [ ]:
3, typeof(3)

In [ ]:
@show bitstring(3); 
@show bitstring(253780027);

Integer arithmetic is exact 

In [ ]:
2 + 3 ==  5

... except when we overflow 

In [ ]:
@show 2^62, bitstring(2^63-1);
@show 2^63, bitstring(2^63);

### Fractions 

This is an interesting subject, especially in Julia, but we don't have time to cover it, so I will skip it. 

In [ ]:
3 // 5, typeof(3 // 5)

In [ ]:
3//5 * 5//3

The basic problem with fraction is that if we perform many operations then they get longer and longer to store. A few or even 10s of operations might be ok, but we will deal with 1000s to millions of operations. We either get overflow or need complicated and compuational expensive schemes to store those big fractions. (see `?big`)

In [ ]:
3//5 * 13/17 * 29/57 * 11/56 * 37 / 87

### Floating Point Numbers

General real numbers cannot be represented

In [ ]:
@show x = 5/7
@show y = (x / 30) * 210;

In [ ]:
@show y == 5    # y should be 5
@show y ≈ 5     # but it is only approximately equal to 5 

In [ ]:
# or even simpler 
@show (0.1 + 0.2) + 0.3
@show 0.1 + (0.2 + 0.3)

The advantage of this is that we can use numbers that aren't fractions, e.g., 

In [ ]:
sqrt(2)

In [ ]:
x = sqrt(2)
@show x^2

In [ ]:
# To help us compare floating point numbers Julia gives us the ≈ operator: 
@show x^2 ≈ 2

### 32 bit Floating point number 

$$ 
    x = s \cdot m \times 2^e
$$
where $s \in \{1, -1\}$ is the sign, $m > 0$ the mantissa (fraction) and $e > 0$ the exponent.

![](https://upload.wikimedia.org/wikipedia/commons/d/d2/Float_example.svg)

E.g., 
$$ 
    12.2792 = 0.122792 \times 10^2
$$
(except of course in base 2 rather than base 10)

In [ ]:
bitstring(Float32(12.2792))

### Floating point operations 

Crudely, we can think of a floating point operation ${\rm fl}(a \odot b)$ as follows: 
1. exact operation $c = a \odot b$ 
2. rounding $d = {\rm round}(d)$

In [ ]:
@show a = Float32(sqrt(2))
@show b = Float32(exp(1))
# "exact" operation by going to more accurate arithmetic
@show c = Float64(a) + Float64(b)
@show d = a + b 

There is much more we should say about floating point arithmetic, but we will keep it to these basics, just a few more comments: 
* Standard accuracy is 64 bit, but sometimes 32bit are use for speed.
* Floating point numbers have a range, i.e. one can underflow or overflow
* Floating point numbers are roughly logarithmically distributed in that range, i.e. relative floating point accuracy is the same at all scales (within the range)

In [ ]:
@show floatmin(Float64), floatmax(Float64)   # the range
@show floatmin(Float32), floatmax(Float32)   # the range

### The Standard Model of Floating Point Arithmetic

Let $\odot$ denote one of the four arithmetic operations, $+, -, \times, \div$. Then in the standard model for floating point arithmetic is to assume that 
$$ 
    {\rm fl}[ a \odot b ] = (a \odot b) (1+\eta)
$$
where $|\eta| \leq \epsilon$ and $\epsilon > 0$ is *machine precision*.

In [ ]:
@show eps(Float64); 
@show eps(Float32);

### Example: finite difference formulas

In exact Arithmetic: 
$$ 
    \frac{f(x+h) - f(x)}{h} = f'(x) + O(h)
$$

In floating point arithmetic: 
$$\begin{aligned}
    {\rm fl}\Big[ \frac{f(x+h) - f(x)}{h}\Big]
    &=  
    \frac{ {\rm fl}[f(x+h) - f(x)] }{h} (1+\eta_1) \\ 
%     &= 
%     \frac{(f(x+h) - f(x))(1+\eta_2)}{h} (1+\eta_1) \\ 
    &= 
    \frac{{\rm fl}[f(x+h)] - {\rm fl}[f(x)]}{h} \frac{1+\eta_2}{1+\eta_1} \\             
    &= 
    \frac{f(x+h)(1+\eta_3) - f(x)(1+\eta_4)}{h} \frac{1+\eta_2}{1+\eta_1} \\   
    &= 
    \frac{f(x+h) - f(x)}{h}  \frac{1+\eta_2}{1+\eta_1} 
    +  \frac{f(x+h) \eta_3 - f(x) \eta_4}{h} \frac{1+\eta_2}{1+\eta_1} \\   
    &=
    \frac{f(x+h) - f(x)}{h} (1+O(\epsilon)) + O(\epsilon/h)
\end{aligned}$$

Summary: 
$$ 
{\rm fl}\Big[ \frac{f(x+h) - f(x)}{h}\Big] = f'(x) + O(h) + O(\epsilon/h)
$$

In [ ]:
using Plots 
hh = 0.5.^(4:45)
f = x -> cos(x); df = x -> - sin(x); x0 = pi/4  # what happens if we change pi/4 -> pi/2 ??
err_ex = [ abs((f(big(x0+h)) - f(big(x0)))/big(h) - df(x0)) for h in hh ]
err_fl = [ abs((f(x0+h) - f(x0))/h - df(x0)) for h in hh ]
plot(hh, err_ex, xaxis = :log, xlabel = "h", yaxis = :log, ylabel = "err", lw = 2, label = "exact", size = (700, 300), ylims = [1e-15, 1e-2], legend = :outertopright)
plot!(hh, err_fl, label = "Float64", lw = 2)

### Things can get arbitrarily bad ... 

The following incredible example is due to [Stefan Karpinski](https://discourse.julialang.org/t/array-ordering-and-naive-summation/1929).  (Julia co-developer, but the example has nothing to do with Julia)


In [ ]:
# The sumsto function always returns the same 2046 floating-point numbers but 
# returns them in a different order based on x
function sumsto(x::Float64)
    0 <= x < exp2(970) || throw(ArgumentError("sum must be in [0,2^970)"))
    n, p₀ = Base.decompose(x) # integers such that `n*exp2(p₀) == x`
    [floatmax(); [exp2(p) for p in -1074:969 if iseven(n >> (p-p₀))]
    -floatmax(); [exp2(p) for p in -1074:969 if isodd(n >> (p-p₀))]]
end

In [ ]:
X1 = sumsto(3.142592654)    #  pi
X2 = sumsto(2.99792458e8)   #  speed of light
@show sort(X1) == sort(X2)  #  the two arrays X1, x2 contain the same numbers!
@show sum(x for x in X1)    #  but sum to entirely different values 
@show sum(x for x in X2);   #    (i.e. order of summation matters!)
@show sum(X1), sum(X2)      # Note that a plain `sum` is numerically more robust! (cf compensated summation)

 .... which is why numerical stability is so important!  If  you think this was a fun example and would like to understand it then look up Kahan summation or compensated summation!

### Numerical Stability in Linear Algebra 

Solving linear systems 
$$
Ax = b
$$ 
is one of the most important everyday tasks in numerical computations. It is therefore paramount that we use numerically stable linear solvers!

We already briefly touched on conditioning which is related.

In [ ]:
using LinearAlgebra

Here is a simple well-conditioned matrix. the resulting linear system is therefore also well-conditioned, i.e. it should be possible to construct the solution in a numerically stable way.

In [ ]:
ϵ = 1e-15
A = [ ϵ    1.0
      1.0  1.0 ]
cond(A) 

In [ ]:
# create a manufactured solution
x = rand(2)
b = A * x;

But if we solve it with an unstable algorithm ... (note that in Julia one has to manually force the usage of an unstable algorithm. The defaults are very good.)

In [ ]:
x2 = lu(A, NoPivot()) \ b 
norm(x2 - x, Inf)

And now with a stable algorithm ... (technically, this uses only partial pivoting, but it appears to suffice almost always!)

In [ ]:
x1 = lu(A) \ b    # same as A \ b
norm(x - x1, Inf)

Another linear system - this time it is ill-conditioned to begin with.

In [ ]:
ϵ = 1e-14
A = [2.0 2.0 - ϵ
     1.0 1.0]
cond(A) 

In [ ]:
x = rand(2); b = A * x 
x1 = lu(A, NoPivot()) \ b 
norm(x1 - x, Inf)

Pivoting has no effect this time...

In [ ]:
x2 = lu(A) \ b 
norm(x2 - x, Inf)

In [ ]:
# but notice that the residual is still small!
norm(A * x1 - b)

#### Backward Stability 

LU Factorisation with Complete Pivoting is Backward Stable. With Partial Pivoting it is stable most of the time. What do we mean by that? 

**Meta-Theorem:** Let $PA = LU$ be the (pivoted) LU factorisation computed in exact arithmetic and let $\tilde{L}, \tilde{U}$ be the LU factors computed in floating point arithmetic. Then there exists a matrix $\tilde{A}$ such that $P\tilde{A} = \tilde{L} \tilde{U}$ and moreover 
$$
    \frac{\| A - \tilde{A} \|}{\|A\|} \leq C \epsilon.
$$

Loosely speaking (not that loosely actually - what is missing?), we can say there exists a perturbed problem 
$$
  \tilde{A} \tilde{x} = b 
$$
which we solve when working with floating point arithmetic. In particular, if $A$ or equivalently $\tilde{A}$ are well-conditioned, then we can use that to demonstrate that the error in the solution will be small as well.

$$\begin{aligned}
   \tilde{A}\tilde{x} &= b, \qquad A x = b \\ 
    A (x - \tilde{x}) &= Ax - A \tilde{x} \\ 
      &= \tilde{A} \tilde{x} - A \tilde{x} \\ 
      &= (\tilde{A} - A) \tilde{x}
\end{aligned}$$

We can deduce 
$$\begin{aligned}
    \|x - \tilde{x} \| &\leq \|A^{-1}\| \| \tilde{A} - A \| \| \tilde{x} \| \\ 
     \frac{\|x - \tilde{x} \|}{\|\tilde{x}\|} &\leq \kappa(A) \frac{\| \tilde{A} - A \|}{\|A\|} \leq C \epsilon
\end{aligned}$$
If the linear system is well-conditioned, then the relative error will be on the order of machine precision. 

Many further refinements to this analysis are of course possible; this is only intended to give you the main idea.


 
### Conclusions

 * Well-conditioned problems still need numerically stable algorithms! 
 * If the problem is ill-conditioned to begin with then no numerics can save us. **BUT** sometimes there is a more hidden way to understand conditioning. It is not always obvious. 
 * More often than not we need not worry about numerical stability, generations of Numerical Analysts have already done this for us!
 * But be aware that the problem does exist and where to start looking in case you run into it!
 * Numerical stability is not always intuitive. E.g. how can naive summation be a problem? Othertimes an expression that looks incredibly unstable turns out to be perfectly fine, e.g. see A2 - barycentric formula.
 * `Float64` is the de-factor standard in numerical computations. 
 * When you really need more accuracy and are willing to sacrifice performance consider using 
     - `Float128` which is implemented in [`QuadMath.jl`](https://github.com/JuliaMath/Quadmath.jl)
     - or `BigFloat` which is built-in and can be accessed via `big`; cf. `?big` for more detail.
     - Use these with great care!!!

#### Further Reading 

* Michael Overton, Numerical Computing with IEEE Floating Point Arithmetic, SIAM 
* Nicholas Higham, Accuracy and Stability of Numerical Algorithms, SIAM